In [191]:
import pandas as pd
import re
import spacy
import unicodedata

In [192]:
nlp = spacy.load("en_core_web_sm")

In [193]:
def clean_name(text: str):
    if not isinstance(text, str):
        return text
    text = text.strip()

    # 1) Xóa prefix "Tạo ..."
    text = re.sub(r"^Tạo\s+", "", text)

    # 2) Xóa hậu tố " - Wikipedia"
    text = re.sub(r"\s+-\s+Wikipedia$", "", text)

    return text

In [194]:
def is_edit_page(link: str):
    """Loại link dạng ?action=edit"""
    if not isinstance(link, str):
        return False
    return "action=edit" in link or "redlink=1" in link


In [195]:
# ============================================
# FIELD (Lĩnh vực nghiên cứu)
# ============================================
FIELD_KEYS = [
    # English
    "physics","physicist","chemist","chemistry","biology","biologist",
    "mathematics","math","mathematician","computer science","informatics",
    "neuroscience","engineering","astronomy","genetics","quantum","statistics",
    "geology","ecology","robotics","ai","artificial intelligence",
    
    # Vietnamese
    "vật lý","nhà vật lý",
    "hóa học","hoá học","nhà hóa học","nhà hoá học",
    "sinh học","nhà sinh học",
    "toán học","nhà toán học",
    "tin học","khoa học máy tính",
    "thiên văn","nhà thiên văn",
    "di truyền","nhà di truyền",
    "kỹ thuật","kĩ thuật","kỹ sư",
    "thống kê","nhà thống kê",
    "địa chất","nhà địa chất",
    "khí tượng","nhà khí tượng"
]


In [196]:
PERSON_LATIN_PATTERN = re.compile(
    r"^[A-Z][a-z]+(?:\s[A-Z][a-z]+){1,2}$"
)

# Tên kiểu Việt Nam hoặc có dấu
PERSON_VI_PATTERN = re.compile(
    r"^[A-ZÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠƯ][\wÀ-ỹ]+(?:\s[A-ZÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠƯ][\wÀ-ỹ]+){1,3}$"
)

# Danh xưng
PERSON_TITLES = ["dr", "prof", "sir", "mr", "mrs", "gs", "ts"]

In [197]:
# ============================================
# OCCUPATION (Nghề nghiệp)
# ============================================
OCCUPATION_KEYS = [
    # English
    "professor","researcher","engineer","doctor","author","writer",
    "inventor","astronaut","teacher","scholar","philosopher","politician",
    "physician","lecturer","academic","scientist",

    # Vietnamese
    "giáo sư","giảng viên","nhà nghiên cứu",
    "nhà phát minh","kỹ sư","kĩ sư",
    "nhà văn","tác giả","nhà triết học","triết gia",
    "chính trị gia","phi hành gia","bác sĩ","nhà khoa học"
]


In [198]:

INSTITUTION_KEYS = [
    # English
    "university","institute","academy","lab","laboratory","college",
    "faculty","department","press","society","center","centre",
    "organization","organisation","research center","research institute",
    "institute of","academy of",

    # Vietnamese
    "đại học","trường đại học",
    "học viện","Viện","Viện nghiên cứu",
    "phòng thí nghiệm","Phòng thí nghiệm quốc gia",
    "khoa","trung tâm","tổ chức","cơ quan","hội",
    "học viện quốc gia","trường","cơ quan","Phòng"
    "nghiên cứu quốc gia","thành viên"
]


In [199]:
# ============================================
# AWARD (Giải thưởng)
# ============================================
AWARD_KEYS = [
    # English
    "prize","medal","award","fellow","distinguished","nobel","laureate",
    "honor","honorary","prêmio",

    # Vietnamese
    "giải thưởng","giải","huy chương","phần thưởng",
    "giải nobel","giải hòa bình","giải vật lý",
    "giải hóa học","giải sinh học"
]


In [200]:
# ============================================
# DATE (Ngày tháng tiếng Anh + tiếng Việt)
# ============================================
DATE_PATTERN = r"(" \
    r"\b\d{4}\b" \
    r"|\b\d{1,2}[-/]\d{1,2}[-/]\d{4}\b" \
    r"|\b\d{1,2} \w+ \d{4}\b" \
    r"|\b\w+ \d{4}\b" \
    r"|\btháng \d{1,2} năm \d{4}\b" \
    r"|\bnăm \d{4}\b" \
    r")"


In [201]:
def classify_unknown(name: str):
    """Chỉ xử lý UNKNOWN, các label khác giữ nguyên."""

    if not isinstance(name, str) or name.strip() == "":
        return "UNKNOWN"

    name_clean = name.strip()

    # ---- DATE ----
    if re.fullmatch(r"\d{3,4}", name_clean):
        return "DATE"
    if re.search(r"\b(19|20)\d{2}\b", name_clean):
        return "DATE"

    # ---- AWARD ----
    if re.search(r"(Nobel|Prize|Medal|Award)", name_clean, re.I):
        return "AWARD"

    # ---- INSTITUTION ----
    if re.search(r"(University|Institute|Academy|College|Laboratory)", name_clean, re.I):
        return "INSTITUTION"

    # ---- FIELD ----
    if re.search(r"(physics|chemistry|biology|mathematics|engineering|astronomy)", name_clean, re.I):
        return "FIELD"

    # ---- PERSON ---- (only if name looks like a person)
    # Pattern: First Last
    if re.fullmatch(r"[A-Z][a-z]+(\s[A-Z][a-z]+){1,3}", name_clean):
        return "PERSON"

    # ---- PERSON using spaCy ----
    doc = nlp(name_clean)
    for ent in doc.ents:
        if ent.label_ in ["PERSON"]:
            return "PERSON"

    return "UNKNOWN"


In [202]:
def process_nodes(input_csv, output_csv):
    df = pd.read_csv(input_csv)

    rows = []
    for _, row in df.iterrows():
        link = row["link"]
        name = row["name"]
        label = row["type"]

        # remove edit pages
        if is_edit_page(link):
            continue

        name2 = clean_name(name)

        # -----------------------------
        # YOUR RULE: EVENT → AWARD
        # -----------------------------
        if label == "EVENT":
            final = "AWARD"

        # -----------------------------
        # ONLY UNKNOWN → heuristic
        # -----------------------------
        elif label == "UNKNOWN":
            final = classify_unknown(name2)

        # keep original for ORG, PER, LOCATION, etc.
        else:
            final = label

        rows.append({
            "link": link,
            "name": name2,
            "type": label,
            "final_label": final
        })

    pd.DataFrame(rows).to_csv(output_csv, index=False)
    print("Saved:", output_csv)


In [203]:
if __name__ == "__main__":
    process_nodes("/kaggle/input/node-cleaned/nodes_raw.csv", "/kaggle/working/nodes_final.csv")


Saved: /kaggle/working/nodes_final.csv


In [204]:
df= pd.read_csv("/kaggle/working/nodes_final.csv")

In [205]:
df

,link,name,type,final_label
0,/wiki/Gi%E1%BA%A3i_Nobel_V%E1%BA%ADt_l%C3%BD,Giải Nobel Vật lý,EVENT,AWARD
1,/wiki/Gi%E1%BA%A3i_Nobel_H%C3%B3a_h%E1%BB%8Dc,Giải Nobel hóa học,EVENT,AWARD
2,/wiki/Gi%E1%BA%A3i_Nobel_Sinh_l%C3%BD_h%E1%BB%...,Giải Nobel Sinh lý học hoặc Y học,EVENT,AWARD
3,/wiki/Gi%E1%BA%A3i_Nobel_V%C4%83n_h%E1%BB%8Dc,Giải Nobel Văn học,EVENT,AWARD
4,/wiki/Gi%E1%BA%A3i_Nobel_H%C3%B2a_b%C3%ACnh,Giải Nobel Hòa bình,EVENT,AWARD
...,...,...,...,...
3899,/wiki/Th%C3%A0nh_vi%C3%AAn_H%E1%BB%99i_Ho%C3%A...,Thành viên Hội Vương thất,PLACE,PLACE
3900,/wiki/%C4%90%E1%BA%A1i_h%E1%BB%8Dc_Ho%C3%A0ng_...,Đại học Nhà vua Luân Đôn,ORGANIZATION,ORGANIZATION
3901,/wiki/Ph%C3%B2ng_Th%C3%AD_nghi%E1%BB%87m_Caven...,Phòng thí nghiệm Cavendish,ORGANIZATION,ORGANIZATION
3902,/wiki/1886,1886,UNKNOWN,DATE


In [206]:
count_unknown = (df["final_label"] == "UNKNOWN").sum()
count_unknown

482

In [207]:
unknown_df = df[df["final_label"] == "UNKNOWN"]
unknown_df



,link,name,type,final_label
6,#cite_note-16,Trang Chính,UNKNOWN,UNKNOWN
12,/wiki/Fr%C3%A9d%C3%A9ric_Passy,Frédéric Passy,UNKNOWN,UNKNOWN
66,/wiki/Ph%C3%B2ng_H%C3%B2a_b%C3%ACnh_Qu%E1%BB%9...,Phòng Hòa bình Quốc tế,UNKNOWN,UNKNOWN
93,/wiki/%E1%BB%A6y_ban_Ch%E1%BB%AF_th%E1%BA%ADp_...,Ủy ban Chữ thập đỏ quốc tế,UNKNOWN,UNKNOWN
187,#endnote_Note1A,Trang Chính,UNKNOWN,UNKNOWN
...,...,...,...,...
3891,/wiki/21_th%C3%A1ng_12,21 tháng 12,UNKNOWN,UNKNOWN
3894,/wiki/9_th%C3%A1ng_8,9 tháng 8,UNKNOWN,UNKNOWN
3897,/wiki/Bradford,Bradford,UNKNOWN,UNKNOWN
3898,/wiki/T%C3%A2y_Yorkshire,Tây Yorkshire,UNKNOWN,UNKNOWN


In [208]:
def norm(text):
    return text.lower().strip()

In [209]:
DATE_PATTERN = r"(\d{1,2}\s+tháng\s+\d{1,2})|(\d{4})"

In [210]:
PERSON_PREFIX = ["ông", "bà", "ngài", "sir", "dr", "gs", "prof"]
LOCATION_SUFFIX = ["shire", "city", "province", "county"]
INSTITUTION_SUFFIX = ["university", "institute", "academy", "committee", "commission"]

In [211]:
FIELD_KEYS = ["physics","chemistry","biology","math","mathematics",
              "neuroscience","genetics","astronomy","engineering"]

OCCUPATION_KEYS = ["scientist","physicist","chemist","biologist",
                   "professor","engineer","researcher"]

INSTITUTION_KEYS = ["university","institute","academy","laboratory",
                    "centre","center","commission","society"]

AWARD_KEYS = ["prize","award","medal","nobel"]


In [212]:
def is_person(name):
    t = norm(name)
    if re.match(r"^[A-Z][a-z]+(\s[A-Z][a-z]+)+$", name):
        return True
    if any(t.startswith(p) for p in PERSON_PREFIX):
        return True
    return False


In [213]:
def is_location(name):
    t = norm(name)
    if t in ["Anh", "Mỹ", "Pháp", "Đức","Ý","Hà Lan","Phần Lan"]:
        return True
    if any(t.endswith(s) for s in LOCATION_SUFFIX):
        return True
    return False


In [214]:
def is_institution(name):
    t = norm(name)
    if any(k in t for k in INSTITUTION_KEYS):
        return True
    if "ủy ban" in t or "tổ chức" in t:
        return True
    return False

In [215]:
def heuristic_unknown(name):
    text = norm(name)

    # bỏ link kiểu "#cite_note-xx"
    if text.startswith("#cite") or text.startswith("#endnote"):
        return "UNKNOWN"

    # DATE
    if re.search(DATE_PATTERN, text):
        return "DATE"

    # PERSON
    if is_person(name):
        return "PERSON"

    # LOCATION
    if is_location(name):
        return "LOCATION"

    # INSTITUTION
    if is_institution(name):
        return "INSTITUTION"

    # AWARD
    if any(k in text for k in AWARD_KEYS):
        return "AWARD"

    # FIELD
    if any(k in text for k in FIELD_KEYS):
        return "FIELD"

    # OCCUPATION
    if any(k in text for k in OCCUPATION_KEYS):
        return "OCCUPATION"

    # Cuối cùng
    return "UNKNOWN"

def fix_label(name, t):
    t = t.upper().strip()

    if t != "UNKNOWN":
        return t

    # fallback
    return heuristic_unknown(name)


In [216]:
def fix_label(name, t):
    t = t.upper().strip()

    # Yêu cầu của bạn: EVENT → AWARD
    if t == "EVENT":
        return "AWARD"

    # Nếu không phải UNKNOWN → giữ nguyên
    if t != "UNKNOWN":
        return t

    # Nếu UNKNOWN → chạy heuristic
    return heuristic_unknown(name)


In [217]:
df = pd.read_csv("/kaggle/working/nodes_final.csv")

if "type" not in df.columns:
    raise Exception(" File phải có cột 'type'!")

df["final_label"] = df.apply(lambda r: fix_label(r["name"], r["type"]), axis=1)

In [218]:
outpath = "/kaggle/working/nodes.csv"
df.to_csv(outpath, index=False)


In [219]:
n = pd.read_csv("/kaggle/working/nodes.csv")
n

,link,name,type,final_label
0,/wiki/Gi%E1%BA%A3i_Nobel_V%E1%BA%ADt_l%C3%BD,Giải Nobel Vật lý,EVENT,AWARD
1,/wiki/Gi%E1%BA%A3i_Nobel_H%C3%B3a_h%E1%BB%8Dc,Giải Nobel hóa học,EVENT,AWARD
2,/wiki/Gi%E1%BA%A3i_Nobel_Sinh_l%C3%BD_h%E1%BB%...,Giải Nobel Sinh lý học hoặc Y học,EVENT,AWARD
3,/wiki/Gi%E1%BA%A3i_Nobel_V%C4%83n_h%E1%BB%8Dc,Giải Nobel Văn học,EVENT,AWARD
4,/wiki/Gi%E1%BA%A3i_Nobel_H%C3%B2a_b%C3%ACnh,Giải Nobel Hòa bình,EVENT,AWARD
...,...,...,...,...
3899,/wiki/Th%C3%A0nh_vi%C3%AAn_H%E1%BB%99i_Ho%C3%A...,Thành viên Hội Vương thất,PLACE,PLACE
3900,/wiki/%C4%90%E1%BA%A1i_h%E1%BB%8Dc_Ho%C3%A0ng_...,Đại học Nhà vua Luân Đôn,ORGANIZATION,ORGANIZATION
3901,/wiki/Ph%C3%B2ng_Th%C3%AD_nghi%E1%BB%87m_Caven...,Phòng thí nghiệm Cavendish,ORGANIZATION,ORGANIZATION
3902,/wiki/1886,1886,UNKNOWN,DATE
